In [ ]:
import numpy as np

In [ ]:
def repressilatorSimulation(alpha_0, alpha, beta, n, showBool):
    doesConverge = False
    
    # Create a model
    model = pints.toy.RepressilatorModel()

    # Run a simulation
    parameters = [alpha_0,alpha,beta,n]
    times = np.linspace(0, 100, 1000)
    values = model.simulate(parameters, times)
    
    peaks, _ = find_peaks(values[:, 0])
    period = np.mean(np.diff(times[peaks]))
    
    # Calculate the amplitude of the oscillation : Naive
    amplitudeNaive = np.max(values[:, 0]) - np.min(values[10:, 0]) # not right at the start
    l = int(len(values) / 2)
    convergesFirstHalf = np.max(values[1:l,0]) - np.min(values[1:l,0])
    convergesSndHalf = np.max(values[l:,0]) - np.min(values[l:,0])

    if (convergesFirstHalf > convergesSndHalf*5 or convergesFirstHalf*5 < convergesSndHalf):
        doesConverge = True

    if (showBool):
        print('Parameters:', parameters)
        print('Period:', period)
        print('AmplitudeNaive:', amplitudeNaive)
        print('Converges = ', doesConverge)

        # Plot the results
        plt.figure()
        plt.xlabel('Time')
        plt.ylabel('Concentration')
        plt.plot(times, values)
        plt.legend(['X', 'Y', 'Z'])
        plt.show()
    return [values, period, amplitudeNaive,doesConverge]

In [ ]:
def noiseAnalysis(alpha_0, alpha, beta, n, showBool):
    originalResult = repressilatorSimulation(alpha_0,alpha,beta,n,False)
    results = []
    for i in range(100):
        parameters = [alpha_0,alpha,beta,n] + 0.1*np.random.randn(4)
        results.append(repressilatorSimulation(*parameters,False))
    # Quantifying Fluctuations in values : NOISE analysis
    peroidFluctuations = np.mean(np.abs(originalResult[1]-[row[1] for row in results]))
    asPercentage = peroidFluctuations / originalResult[1]
    #print(asPercentage) # 6.5% Fluctuations

    amplitudeFluctuations =  np.mean(np.abs(originalResult[2]-[row[2] for row in results]))
    asPercentage2 = amplitudeFluctuations / originalResult[2]
    #print(asPercentage2) # 25.1% Fluctuations
    return [asPercentage, asPercentage2]

In [ ]:
# Parameter Sensitivity Analysis --> what do the input variables do?
# Changing alpha_0
alpha_0 = [0,2,4,8,16,32]
results = np.array([])
for i in range(len(alpha_0)):
    results = np.append(results,repressilatorSimulation(alpha_0[i],1000,5,2,True))


In [ ]:
# Changing alpha
alpha = [10,50,250,1250,6250]
results = np.array([])
for i in range(len(alpha)):
    results = np.append(results,repressilatorSimulation(1,alpha[i],5,2,True))


In [ ]:
# Changing beta
beta = [1,5,25,100,1000]
results = np.array([])
for i in range(len(beta)):
    results = np.append(results,repressilatorSimulation(1,1000,beta[i],2,True))


In [ ]:
# Changing N 
N = [1,2,2.2,3,4]
results = np.array([])
for i in range(len(N)):
    results = np.append(results,repressilatorSimulation(1,1000,5,N[i],True))

In [ ]:
# Convergence Analysis
alpha_0 = [0,2,4,8,16,32]
alpha = [10,50,250,1250,6250]
beta = [1,5,25,100,1000]
N = [1,2,2.2,3,4]
data = np.array(np.meshgrid(alpha_0,alpha,beta,N)).T.reshape(-1,4)
convergenceData = np.array([])
for i in range(len(data)):
    boolean = repressilatorSimulation(data[i][0],data[i][1],data[i][2],data[i][3],False)[3]
    convergenceData = np.append(convergenceData,boolean)

data = np.concatenate((data, convergenceData.reshape(-1, 1)), axis=1)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

countConverging = np.count_nonzero(convergenceData)
dataLen = len(convergenceData)
dataConvergence = data[data[:,4] == 1]
dataNonConvergence = data[data[:,4] == 0]
# Create a sample dataset with 5 rows and 4 columns
df = pd.DataFrame({
    'alpha_0': dataNonConvergence[:,0],
    'alpha': dataNonConvergence[:,1],
    'beta': dataNonConvergence[:,2],
    'N': dataNonConvergence[:,3],
    'Convergence' : dataNonConvergence[:,4]
})

fig = px.parallel_categories(df)

fig.show()

In [ ]:
# Analysis of Convergence
# % of converging vs NonConverging
print('Converging = ',countConverging, '/',dataLen, '=', countConverging/dataLen, '%',
      '\nNonConverging = ' ,(dataLen-countConverging),'/',dataLen, (dataLen-countConverging)/dataLen, '%')

In [ ]:
# Same analysis but for non converging data. use to compare
dff = pd.DataFrame({
    'alpha_0': dataConvergence[:,0],
    'alpha': dataConvergence[:,1],
    'beta': dataConvergence[:,2],
    'N': dataConvergence[:,3],
    'Convergence' : dataConvergence[:,4]
})
fig = px.parallel_categories(dff)

fig.show()

In [ ]:
# Noise analysis for all data, this takes a while
noisePeriod = np.array([])
noiseAmplitude = np.array([])

for i in range(len(dataNonConvergence)):
    resultsPeriod = noiseAnalysis(dataNonConvergence[i,0],dataNonConvergence[i,1],
                                  dataNonConvergence[i,2],dataNonConvergence[i,3],False)[0]
    resultsAmplitude = noiseAnalysis(dataNonConvergence[i,0],dataNonConvergence[i,1],
                                     dataNonConvergence[i,2],dataNonConvergence[i,3],False)[1]
    noisePeriod = np.append(noisePeriod,resultsPeriod)
    noiseAmplitude = np.append(noiseAmplitude,resultsAmplitude)

In [ ]:
import plotly.express as px
dataWperiod = np.concatenate((dataNonConvergence, noisePeriod.reshape(-1, 1)), axis=1)

dff = pd.DataFrame({
    'alpha_0': dataWperiod[:,0],
    'alpha': dataWperiod[:,1],
    'beta': dataWperiod[:,2],
    'N': dataWperiod[:,3],
    'noisePeriod' : dataWperiod[:,5]
})
fig = px.parallel_coordinates(dff, color="noisePeriod", color_continuous_scale=px.colors.diverging.Tealrose,
                             color_continuous_midpoint=2)
fig.show()

In [ ]:
dataWamlitude = np.concatenate((dataNonConvergence, noiseAmplitude.reshape(-1, 1)), axis=1)

dff = pd.DataFrame({
    'alpha_0': dataWperiod[:,0],
    'alpha': dataWperiod[:,1],
    'beta': dataWperiod[:,2],
    'N': dataWperiod[:,3],
    'noiseAmplitude' : dataWperiod[:,5]
})
fig = px.parallel_coordinates(dff, color="noiseAmplitude", color_continuous_scale=px.colors.diverging.Tealrose,
                             color_continuous_midpoint=2)
fig.show()